In [1]:
#### Dependencies

from PIL import Image
from PIL import ImageDraw

import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import sys
import timeit


In [2]:
#### Global variables

model = {}
simplified_model = {}
map_8bit_to_3bit = [i // 32 for i in range(256)]


In [22]:
#### Functions

def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def get_img_colors(img):
    img_colors = list(img.getdata())
            
    return img_colors

def construct_img(img_colors, img_dimension):
    img_new = Image.new('RGB', (img_dimension), (255, 255, 255))
    draw = ImageDraw.Draw(img_new)
    
    img_new_width = img_dimension[0]
    img_new_height = img_dimension[1]
    
    loop_count = 0
    img_colors_len = len(img_colors)

    for w in range(0, img_new_width):
        for h in range(0, img_new_height):
            if(loop_count == img_colors_len):
                break
            
            current_color = (img_colors[loop_count], img_colors[loop_count], img_colors[loop_count])
            draw.rectangle((w, h, w + 1, h + 1), fill=current_color)
            loop_count += 1
    
    return img_new

def get_3bit_colors(img_colors):
    loop_count = 0
    
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = [] 
    loop_count = 0
    
    for row in range(256):
        new_img_matrix_row = []
        for col in range(256):
            new_img_matrix_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(new_img_matrix_row)
        
    return img_matrix

def get_img_features(img, angle, distance):
    img_colors = get_img_colors(img)
    img_matrix = get_img_matrix(img_colors)
    
    glcm_matrix = [[0 for j in range(256)] for i in range(256)] 
    non_zero_glcm_indexes = []
    
    total_pair = 0
    loop_count = 0
    row_id = 0
    for row in img_matrix:
        col_id = 0
        for col in row:
            current_color = col
            neighbor_color = 999
            
            try:
                if(angle == 0):
                    neighbor_color = img_matrix[row_id][col_id + distance]
                elif(angle == 45):
                    neighbor_color = img_matrix[row_id - distance][col_id + distance]
                elif(angle == 90):
                    neighbor_color = img_matrix[row_id - distance][col_id]
                elif(angle == 135):
                    neighbor_color = img_matrix[row_id - distance][col_id - distance]
                    
                glcm_matrix[current_color][neighbor_color] += 1
                total_pair += 1
                
                current_index = (row_id, col_id)
                if(not current_index in non_zero_glcm_indexes):
                    non_zero_glcm_indexes.append(current_index)
            except Exception:
                pass            
            
            col_id += 1
            
        row_id += 1
        
    # Contrast
    img_contrast = 0
    
    # Correlation
    avg_row_glcm = 0
    avg_col_glcm = 0
    
    # Energy
    img_energy = 0
    
    # Homogenity
    img_homogenity = 0
    
    for non_zero_glcm_index in non_zero_glcm_indexes:
        non_zero_glcm_row_id = non_zero_glcm_index[0]
        non_zero_glcm_col_id = non_zero_glcm_index[1]
        
        glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id] /= total_pair
        
        # calculate image's contrast
        img_contrast += ((non_zero_glcm_row_id - non_zero_glcm_col_id) ** 2) * glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id]
        
        # calculate image's correlation
        avg_row_glcm += non_zero_glcm_row_id * glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id]
        avg_col_glcm += non_zero_glcm_col_id * glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id]
        
        # calculate image's energy
        img_energy += glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id] ** 2
        
        # calculate image's homogenity
        img_homogenity += glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id] / (1 + abs(non_zero_glcm_row_id - non_zero_glcm_col_id))
        
    # Correlation
    sigma_row_glcm = 0
    sigma_col_glcm = 0
        
    for non_zero_glcm_index in non_zero_glcm_indexes:
        non_zero_glcm_row_id = non_zero_glcm_index[0]
        non_zero_glcm_col_id = non_zero_glcm_index[1]
        
        # calculate image's correlation
        sigma_row_glcm += ((non_zero_glcm_row_id - avg_row_glcm) ** 2) * glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id]
        sigma_col_glcm += ((non_zero_glcm_col_id - avg_col_glcm) ** 2) * glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id]
        
    # Correlation
    sigma_row_glcm = math.sqrt(sigma_row_glcm)
    sigma_col_glcm = math.sqrt(sigma_col_glcm)
    img_correlation = 0
    
    if(sigma_row_glcm == 0 or sigma_col_glcm == 0):
        sigma_row_glcm = 1
        sigma_col_glcm = 1
    
    for non_zero_glcm_index in non_zero_glcm_indexes:
        non_zero_glcm_row_id = non_zero_glcm_index[0]
        non_zero_glcm_col_id = non_zero_glcm_index[1]
        
        # calculate image's correlation
        img_correlation += ((non_zero_glcm_row_id - avg_row_glcm) * (non_zero_glcm_col_id - avg_col_glcm) * glcm_matrix[non_zero_glcm_row_id][non_zero_glcm_col_id]) / (sigma_row_glcm * sigma_col_glcm)
    
    glcm_values = (img_contrast, img_correlation, img_energy, img_homogenity)
    
    return glcm_values

# Modelling

def initialize_model():
    global model
    global simplified_model
    
    model = {}
    simplified_model = {}
    
def get_class_names(training_folder_path):
    return os.listdir(training_folder_path)

def add_classes_to_model(class_names):
    for class_name in class_names:
        if not class_name in model:
            model[class_name] = []
        if not class_name in simplified_model:
            simplified_model[class_name] = (0, 0, 0, 0)
            
def add_features_to_model(class_name, features):
    if(class_name in model):
        model[class_name].append(features)

# Training
        
def train(training_folder_path, img_type, glcm_angle, glcm_distance):
    class_names = get_class_names(training_folder_path)
    for class_name in class_names:
        training_img_paths = glob.glob(training_folder_path + class_name + '/' + img_type)

        for training_img_path in training_img_paths:
            #print('Training on ' + training_img_path)
            training_img = load_img(training_img_path)

            training_img_class_name = class_name
            training_img_glcm_features = get_img_features(training_img, glcm_angle, glcm_distance)

            add_features_to_model(training_img_class_name, training_img_glcm_features)
        
        simplified_model[class_name] = tuple(np.median(model[class_name], axis=0))
        
# Classification

def get_img_centroid_distance(img_features, centroid_features, gclm_filter, glcm_formula):
    img_formula_cache = [0, 0, 0, 0]
    centroid_formula_cache = [0, 0, 0, 0]
    current_formula_cache_index = 0
    loop_count = 0
    
    for char in glcm_formula:
        try:
            glcm_index = int(char)
            img_formula_cache[current_formula_cache_index] += img_features[glcm_index] * glcm_filter[glcm_index]
            centroid_formula_cache[current_formula_cache_index] += centroid_features[glcm_index] * glcm_filter[glcm_index]
            #01, 23
        except Exception:
            if(char == ','):
                current_formula_cache_index += 1
            
        loop_count += 1
        
    img_centroid_distance_before_sqrt = 0
    formula_cache_constraint = current_formula_cache_index + 1
    for formula_cache_index in range(formula_cache_constraint):
        img_centroid_distance_before_sqrt += (img_formula_cache[formula_cache_index] - centroid_formula_cache[formula_cache_index]) ** 2
        
    img_centroid_distance = math.sqrt(img_centroid_distance_before_sqrt)
    
    return img_centroid_distance
            
    #(contrast_correlation1, energy_homogenity1) = img_features
    #(contrast_correlation2, energy_homogenity2) = centroid_features
    
    #return math.sqrt(((contrast_correlation2 - contrast_correlation1) ** 2) + ((energy_homogenity2 - energy_homogenity1) ** 2))
    
def get_img_class(img, k_neighbors, glcm_angle, glcm_distance, glcm_filter, glcm_formula):
    img_glcm_features = get_img_features(img, glcm_angle, glcm_distance)
    
    #kNN
    min_img_centroid_distance = [0 for i in range(k_neighbors)]
    img_class_names = ['unknown' for i in range(k_neighbors)]
    is_first_loop = True
    for class_name in model:
        for class_centroid_features in model[class_name]:
            img_centroid_distance = get_img_centroid_distance(img_glcm_features, class_centroid_features, glcm_filter, glcm_formula)
        
            if(is_first_loop):
                min_img_centroid_distance[0] = img_centroid_distance
                img_class_names[0] = class_name
                is_first_loop = False
            else:
                if(img_centroid_distance < min_img_centroid_distance[0]):
                    for i in range(k_neighbors - 1, 0, -1):
                        min_img_centroid_distance[i] = min_img_centroid_distance[i-1]
                        img_class_names[i] = img_class_names[i-1]

                    min_img_centroid_distance[0] = img_centroid_distance
                    img_class_names[0] = class_name
    
    #print('The image categorized as ' + img_class_name + ' class.')
    #print(img_glcm_features)
    #print_img(img)    
    
    img_class_names = list(filter(lambda val: val != 'unknown', img_class_names))    
    img_class_name = max(set(img_class_names), key=img_class_names.count)
        
    return img_class_name

# Validation

def validate(validation_folder_path, img_type, k_neighbors, glcm_angle, glcm_distance, glcm_filter, glcm_formula):
    right_answer = 0
    total_validation = 0
    
    class_names = os.listdir(validation_folder_path)
    for class_name in class_names:
        validation_img_paths = glob.glob(validation_folder_path + class_name + '/' + img_type)

        for validation_img_path in validation_img_paths:
            #print('Validating on ' + validation_img_path)
            validation_img = load_img(validation_img_path)
            
            validation_img_class_name = get_img_class(validation_img, k_neighbors, glcm_angle, glcm_distance, glcm_filter, glcm_formula)
            expected_img_class_name = class_name
            
            if(validation_img_class_name == expected_img_class_name):
                right_answer += 1
            
            total_validation += 1
    
    accuracy = (right_answer / total_validation) * 100
    
    print('Accuracy = ' + str(accuracy) + '%')
    
    return accuracy

In [14]:
img = load_img('./test/cap2.jpg')
print(len(list(img.getdata())))

275200


In [ ]:
#### Main

#Calculate time elapsed for a model
time_modelling_start = timeit.default_timer()

# Defining paths
root_path = './'
training_folder_path = root_path + 'training/'
validation_folder_path = root_path + 'validation/'
test_folder_path = root_path + 'test/'

# Defining model specifications
img_type = '*.jpg'
k_neighbors = 1
glcm_angle = 0
glcm_distance = 1
glcm_filter = (1, 1, 1, 1)
glcm_formula = '0, 1, 2, 3'

# Initializing
initialize_model()
class_names = get_class_names(training_folder_path)
add_classes_to_model(class_names)

# Training
train(training_folder_path, img_type, glcm_angle, glcm_distance)

# Validating
model_accuracy = validate(validation_folder_path, img_type, k_neighbors, glcm_angle, glcm_distance, glcm_filter, glcm_formula)

#Calculate time elapsed for a model
time_modelling_stop = timeit.default_timer()

# Testing
test_img_path = test_folder_path + 'cap2.jpg'
test_img = load_img(test_img_path)
test_img_class = get_img_class(test_img, k_neighbors, glcm_angle, glcm_distance, glcm_filter, glcm_formula)
print('This image is categorized as batik ' + test_img_class + '.')
print_img(test_img)

#Calculate time elapsed for a classification
time_classification_end = timeit.default_timer()


#Print total time elapsed for a model
print('\n\n')
print('Model specifications')
print('====================')
print('Image type         : ' + img_type)
print('K-neighbors        : ' + str(k_neighbors))
print('GLCM Angle (deg)   : ' + str(glcm_angle))
print('GLCM distance      : ' + str(glcm_distance))
print('GLCM filter        : ' + str(glcm_filter))
print('GLCM formula       : ' + str(glcm_formula))
print()
print('Model\'s accuracy is around ' + str(model_accuracy) + '%')
print('Time elapsed to make this model is ' + str(time_modelling_stop - time_modelling_start) + ' seconds.')
print()
print('Time elapsed to classify one image is ' + str(time_classification_end - time_modelling_stop) + ' seconds.')